In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Following the Standard Pattern to Solve a Data Science Problem
"""
Step 1: Define the Problem
Project Summary: Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. 
We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, 
the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided 
with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. 
Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly 
using records recovered from the spaceship’s damaged computer system.

Help save them and change history!"""

"""
Step 2: Gather the Data
The dataset is also given to us with test and train data at Spaceship Titanic
"""

"""
Step 3: Prepare Data for Consumption
Since step 2 was provided to us, so is step 3. 
Therefore, normal processes in data wrangling, such as data architecture, governance, and extraction are out of scope. 
Thus, only data cleaning is in scope.
"""


In [ ]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.4)
import itertools
import warnings
warnings.filterwarnings('ignore')
import time

# Sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
#from sklearn.metrics import roc_auc_score, plot_roc_curve, roc_curve
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.utils import resample

# Models
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier


#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [ ]:
# Save to df
pd.set_option('display.max_columns', None)
train = pd.read_csv('../input/spaceship-titanic/train.csv')
test = pd.read_csv('../input/spaceship-titanic/test.csv')

# Shape and preview
print('Train set shape:', train.shape)
print('Test set shape:', test.shape)
train.head()

In [ ]:
#Checking for duplicates

train.duplicated().sum(), test.duplicated().sum()

In [ ]:
#EDA
# Figure size

plt.figure(figsize=(10, 6))

# Countplot
sns.countplot(x='Transported', data=train, palette='viridis')
total = len(train['Transported'])
for p in plt.gca().patches:
    height = p.get_height()
    plt.text(p.get_x() + p.get_width() / 2., height + 0.1,
             '{:.1%}'.format(height/total),
             ha="center", fontsize=12)

plt.title("Target distribution")
plt.show()

"We have a balanced target value - No need for Over/Undersampling"

In [ ]:
#Checking for all na values

train.isna().sum()

In [ ]:
#Continuous features
# Figure size
plt.figure(figsize=(10,4))
sns.histplot(data=train, x='Age', hue='Transported', binwidth=1, kde=True)
plt.title('Age distribution')
plt.xlabel('Age (years)')

"""
0-18 year olds were more likely to be transported than not.
18-25 year olds were less likely to be transported than not.
Over 25 year olds were about equally likely to be transported than not.
"""

In [ ]:
train['Age'].skew()
#Perfect Skew = 0
#Moderate Skew is between -0.5 to +0.5 ie fairly symmetric and can be considered norm dist

In [ ]:
# New features - training set
train['Age_group']=np.nan
train.loc[train['Age']<=12,'Age_group']='Age_0-12'
train.loc[(train['Age']>12) & (train['Age']<18),'Age_group']='Age_13-17'
train.loc[(train['Age']>=18) & (train['Age']<=25),'Age_group']='Age_18-25'
train.loc[(train['Age']>25) & (train['Age']<=30),'Age_group']='Age_26-30'
train.loc[(train['Age']>30) & (train['Age']<=50),'Age_group']='Age_31-50'
train.loc[train['Age']>50,'Age_group']='Age_51+'

# New features - test set
test['Age_group']=np.nan
test.loc[test['Age']<=12,'Age_group']='Age_0-12'
test.loc[(test['Age']>12) & (test['Age']<18),'Age_group']='Age_13-17'
test.loc[(test['Age']>=18) & (test['Age']<=25),'Age_group']='Age_18-25'
test.loc[(test['Age']>25) & (test['Age']<=30),'Age_group']='Age_26-30'
test.loc[(test['Age']>30) & (test['Age']<=50),'Age_group']='Age_31-50'
test.loc[test['Age']>50,'Age_group']='Age_51+'

# Plot distribution of new features
plt.figure(figsize=(10,4))
g=sns.countplot(data=train, x='Age_group', hue='Transported', order=['Age_0-12','Age_13-17','Age_18-25','Age_26-30','Age_31-50','Age_51+'])
plt.title('Age group distribution')

In [ ]:
exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
# New features - training set
train['Expenditure']=train[exp_feats].sum(axis=1)
train['No_spending']=(train['Expenditure']==0).astype(int)

# New features - test set
test['Expenditure']=test[exp_feats].sum(axis=1)
test['No_spending']=(test['Expenditure']==0).astype(int)

# Plot distribution of new features
fig=plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(data=train, x='Expenditure', hue='Transported', bins=200)
plt.title('Total expenditure (truncated)')
plt.ylim([0,200])
plt.xlim([0,20000])

plt.subplot(1,2,2)
sns.countplot(data=train, x='No_spending', hue='Transported')
plt.title('No spending indicator')
fig.tight_layout()

In [ ]:
# New feature - Group
train['Group'] = train['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
test['Group'] = test['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)

# New feature - Group size ranging from 1-8
train['Group_size']=train['Group'].map(train['Group'].value_counts())
test['Group_size']=test['Group'].map(test['Group'].value_counts())

# Plot distribution of new features
plt.figure(figsize=(20,4))
plt.subplot(1,2,1)
sns.histplot(data=train, x='Group', hue='Transported', binwidth=1)
plt.title('Group')

plt.subplot(1,2,2)
sns.countplot(data=train, x='Group_size', hue='Transported')
plt.title('Group size')
fig.tight_layout()

In [ ]:
# New feature
train['Solo']=(train['Group_size']==1).astype(int)
test['Solo']=(test['Group_size']==1).astype(int)

# New feature distribution
plt.figure(figsize=(10,4))
sns.countplot(data=train, x='Solo', hue='Transported')
plt.title('Passenger travelling solo or not')
plt.ylim([0,3000])


"""
Person travelling Solo have high chance of NOT reaching the destination compare to people travelling in groups
"""

In [ ]:
# Create another figure
plt.figure(figsize=(10,6))

# Start with positve examples
plt.scatter(train.Age[train['Transported']==True], 
            train.Expenditure[train['Transported']==True], 
            c="salmon") # define it as a scatter figure

# Now for negative examples, we want them on the same plot, so we call plt again
plt.scatter(train.Age[train['Transported']==False], 
            train.Expenditure[train['Transported']==False], 
            c="lightblue") # axis always come as (x, y)

# Add some helpful info
plt.title("Age Vs Expenditure")
plt.xlabel("Age")
plt.legend(["Not Transported", "Transported"])
plt.ylabel("Expenditure");

In [ ]:
#Categorical features
# Categorical features
cat_feats=['HomePlanet', 'CryoSleep', 'Destination', 'VIP']

# Plot categorical features
fig=plt.figure(figsize=(10,16))
for i, var_name in enumerate(cat_feats):
    ax=fig.add_subplot(4,1,i+1)
    sns.countplot(data=train, x=var_name, axes=ax, hue='Transported')
    ax.set_title(var_name)
fig.tight_layout()  # Improves appearance a bit
plt.show()

In [ ]:
train[train['VIP'].isin([True])]

In [ ]:
#Checking VIP count
#EDA
# Figure size

plt.figure(figsize=(10, 6))
# Countplot
sns.countplot(x='VIP', data=train, palette='viridis')
total = len(train['VIP'])
for p in plt.gca().patches:
    height = p.get_height()
    plt.text(p.get_x() + p.get_width() / 2., height + 0.1,
             '{:.1%}'.format(height/total),
             ha="center", fontsize=12)

plt.title("VIP distribution")
plt.show()

"""95% people are no VIP"""

In [ ]:
train.head()

In [ ]:
vte = train.groupby(['VIP', 'Transported'])['Expenditure'].count().unstack()


# Heatmap of missing values
plt.figure(figsize=(10,4))
sns.heatmap(vte.T, annot=True, fmt='g', cmap='coolwarm')

#We can consider removing VIP feature 

In [ ]:
#Cabin location
#Extract deck, number and side from cabin feature. 

# Replace NaN's with outliers for now (so we can split feature)
train['Cabin'].fillna('Z/9999/Z', inplace=True)
test['Cabin'].fillna('Z/9999/Z', inplace=True)

# New features - training set
train['Cabin_deck'] = train['Cabin'].apply(lambda x: x.split('/')[0])
train['Cabin_number'] = train['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
train['Cabin_side'] = train['Cabin'].apply(lambda x: x.split('/')[2])

# New features - test set
test['Cabin_deck'] = test['Cabin'].apply(lambda x: x.split('/')[0])
test['Cabin_number'] = test['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
test['Cabin_side'] = test['Cabin'].apply(lambda x: x.split('/')[2])

# Put Nan's back in (we will fill these later)
train.loc[train['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
train.loc[train['Cabin_number']==9999, 'Cabin_number']=np.nan
train.loc[train['Cabin_side']=='Z', 'Cabin_side']=np.nan
test.loc[test['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
test.loc[test['Cabin_number']==9999, 'Cabin_number']=np.nan
test.loc[test['Cabin_side']=='Z', 'Cabin_side']=np.nan

# Drop Cabin (we don't need it anymore)
train.drop('Cabin', axis=1, inplace=True)
test.drop('Cabin', axis=1, inplace=True)

# Plot distribution of new features
fig=plt.figure(figsize=(10,12))
plt.subplot(3,1,1)
sns.countplot(data=train, x='Cabin_deck', hue='Transported', order=['A','B','C','D','E','F','G','T'])
plt.title('Cabin deck')

plt.subplot(3,1,2)
sns.histplot(data=train, x='Cabin_number', hue='Transported',binwidth=20)
plt.vlines(300, ymin=0, ymax=200, color='black')
plt.vlines(600, ymin=0, ymax=200, color='black')
plt.vlines(900, ymin=0, ymax=200, color='black')
plt.vlines(1200, ymin=0, ymax=200, color='black')
plt.vlines(1500, ymin=0, ymax=200, color='black')
plt.vlines(1800, ymin=0, ymax=200, color='black')
plt.title('Cabin number')
plt.xlim([0,2000])

plt.subplot(3,1,3)
sns.countplot(data=train, x='Cabin_side', hue='Transported')
plt.title('Cabin side')
fig.tight_layout()

#Cabin T can be removed

In [ ]:
train.head()

In [ ]:
# New features - training set
train['Cabin_region1']=(train['Cabin_number']<300).astype(int)   # one-hot encoding
train['Cabin_region2']=((train['Cabin_number']>=300) & (train['Cabin_number']<600)).astype(int)
train['Cabin_region3']=((train['Cabin_number']>=600) & (train['Cabin_number']<900)).astype(int)
train['Cabin_region4']=((train['Cabin_number']>=900) & (train['Cabin_number']<1200)).astype(int)
train['Cabin_region5']=((train['Cabin_number']>=1200) & (train['Cabin_number']<1500)).astype(int)
train['Cabin_region6']=((train['Cabin_number']>=1500) & (train['Cabin_number']<1800)).astype(int)
train['Cabin_region7']=(train['Cabin_number']>=1800).astype(int)

# New features - test set
test['Cabin_region1']=(test['Cabin_number']<300).astype(int)   # one-hot encoding
test['Cabin_region2']=((test['Cabin_number']>=300) & (test['Cabin_number']<600)).astype(int)
test['Cabin_region3']=((test['Cabin_number']>=600) & (test['Cabin_number']<900)).astype(int)
test['Cabin_region4']=((test['Cabin_number']>=900) & (test['Cabin_number']<1200)).astype(int)
test['Cabin_region5']=((test['Cabin_number']>=1200) & (test['Cabin_number']<1500)).astype(int)
test['Cabin_region6']=((test['Cabin_number']>=1500) & (test['Cabin_number']<1800)).astype(int)
test['Cabin_region7']=(test['Cabin_number']>=1800).astype(int)

# Plot distribution of new features - giving importance to subsequent cabin region values
plt.figure(figsize=(10,4))
train['Cabin_regions_plot']=(train['Cabin_region1']+2*train['Cabin_region2']+3*train['Cabin_region3']+4*train['Cabin_region4']+5*train['Cabin_region5']+6*train['Cabin_region6']+7*train['Cabin_region7']).astype(int)
sns.countplot(data=train, x='Cabin_regions_plot', hue='Transported')
plt.title('Cabin regions')
train.drop('Cabin_regions_plot', axis=1, inplace=True)

In [ ]:
#As the space-ship titanic event is happening, cabin region based on cabin number is an important parameter 
# as when the disaster happens, cabin region will play and important role deciding if the passenger will be transported or NOT

In [ ]:
#Last name - Same last name would be same HomePlanet, same group
#Calculate family size from last name.

# Replace NaN's with outliers for now (so we can split feature)
train['Name'].fillna('Unknown Unknown', inplace=True)
test['Name'].fillna('Unknown Unknown', inplace=True)

# New feature - Surname
train['Surname']=train['Name'].str.split().str[-1]
test['Surname']=test['Name'].str.split().str[-1]

# New feature - Family size
train['Family_size']=train['Surname'].map(test['Surname'].value_counts())
test['Family_size']=test['Surname'].map(test['Surname'].value_counts())

# Put Nan's back in (we will fill these later) - handling improper dataset
train.loc[train['Surname']=='Unknown','Surname']=np.nan
train.loc[train['Family_size']>100,'Family_size']=np.nan

test.loc[test['Surname']=='Unknown','Surname']=np.nan
test.loc[test['Family_size']>100,'Family_size']=np.nan

# Drop name (we don't need it anymore)
train.drop('Name', axis=1, inplace=True)
test.drop('Name', axis=1, inplace=True)

# New feature distribution
plt.figure(figsize=(12,4))
sns.countplot(data=train, x='Family_size', hue='Transported')
plt.title('Family size')

In [ ]:
#EDA Over, Time to Clean the data

In [ ]:
train.head()

In [ ]:
train.pivot_table(index='Group',columns='HomePlanet', values='Expenditure', aggfunc='count')

In [ ]:
#Missing Values

#Combine train and test
# Labels and features
y=train['Transported'].copy().astype(int)
X=train.drop('Transported', axis=1).copy()

# Concatenate dataframes
data=pd.concat([X, test], axis=0).reset_index(drop=True)

In [ ]:
#Explore missing values
# Columns with missing values
na_cols=data.columns[data.isna().any()].tolist()

# Missing values summary
mv=pd.DataFrame(data[na_cols].isna().sum(), columns=['Number_missing'])
mv['Percentage_missing']=np.round(100*mv['Number_missing']/len(data),2)
mv

In [ ]:
#Dealing with missing values 1 at a time

In [ ]:
data['Group'].value_counts()

In [ ]:
data.head()

In [ ]:
pd.crosstab(data['Group'], data['HomePlanet'])

#at group 3, europa is repeated 2 twice which means group 3 has 2 people both from Europa, 
    #therefore in crosstab, at group 3, europa is has 2

In [ ]:
# Joint distribution of Group and HomePlanet
GHP_gb=data.groupby(['Group','HomePlanet'])['HomePlanet'].count().unstack().fillna(0)


"""
Everyone in the same group comes from the same home planet. 
So we can fill the missing HomePlanet values according to the group.
(At least the ones where the group size is bigger than 1.)
"""

GHP_gb

In [ ]:
#Finding group of similiar people in GROUP and takes its index value,
#replace nan at HomePlanet with people of same group!

# Missing values before
HP_bef=data['HomePlanet'].isna().sum()

# Passengers with missing HomePlanet and in a group with known HomePlanet
GHP_index=data[data['HomePlanet'].isna()][(data[data['HomePlanet'].isna()]['Group']).isin(GHP_gb.index)].index
#GHP_index = data[data['HomePlanet'].isna() & data['Group'].isin(GHP_gb.index)].index

# Fill corresponding missing values
data.loc[GHP_index,'HomePlanet']=data.iloc[GHP_index,:]['Group'].map(lambda x: GHP_gb.idxmax(axis=1)[x])
#data.loc[GHP_index, 'HomePlanet'] = data.loc[GHP_index, 'Group'].replace(GHP_gb.idxmax(axis=1))

# Print number of missing values left
print('#HomePlanet missing values before:',HP_bef)
print('#HomePlanet missing values after:',data['HomePlanet'].isna().sum())

In [ ]:
data.head()
#HomePlanet and Group Done

In [ ]:
cshp = data.groupby(['CryoSleep','HomePlanet'])['HomePlanet'].count().unstack().fillna(0)
plt.figure(figsize=(10,4))
sns.heatmap(cshp.T, annot=True, fmt='g', cmap='coolwarm')

#It would be risky to use CryoSleep to fill missing HomePlanet as it is binary and it also has missing values

In [ ]:
data.head()

In [ ]:
#Using other features to fill missing values- cabin deck
# Joint distribution of CabinDeck and HomePlanet
CDHP_gb=data.groupby(['Cabin_deck','HomePlanet'])['HomePlanet'].size().unstack().fillna(0)

# Heatmap of missing values
plt.figure(figsize=(10,4))
sns.heatmap(CDHP_gb.T, annot=True, fmt='g', cmap='coolwarm')

"""
Passengers on decks A, B, C , T came from Europa.
Passengers on deck G came from Earth.
Passengers on decks D, E or F came from multiple planets.
"""

#As D,E,F has multiple values, we won't use use to find missing HomePlanet

In [ ]:
# Missing values before
HP_bef=data['HomePlanet'].isna().sum()

# Decks A, B, C or T came from Europa
data.loc[(data['HomePlanet'].isna()) & (data['Cabin_deck'].isin(['A', 'B', 'C', 'T'])), 'HomePlanet']='Europa'

# Deck G came from Earth
data.loc[(data['HomePlanet'].isna()) & (data['Cabin_deck']=='G'), 'HomePlanet']='Earth'

# Print number of missing values left
print('#HomePlanet missing values before:',HP_bef)
print('#HomePlanet missing values after:',data['HomePlanet'].isna().sum())

In [ ]:
# Joint distribution of Surname and HomePlanet - Using Surname
SHP_gb=data.groupby(['Surname','HomePlanet'])['HomePlanet'].size().unstack().fillna(0)

# Countplot of unique values
plt.figure(figsize=(10,4))
sns.countplot((SHP_gb>0).sum(axis=1))
plt.title('Number of unique planets per surname') 
#Everyone with the same surname comes from the same home planet.

In [ ]:
SHP_gb

In [ ]:
# Missing values before
HP_bef=data['HomePlanet'].isna().sum()


# SHP_index=data[data['HomePlanet'].isna()][(data[data['HomePlanet'].isna()]['Surname']).isin(SHP_gb.index)].index

# # Fill corresponding missing values
# data.loc[SHP_index,'HomePlanet']=data.iloc[SHP_index,:]['Surname'].map(lambda x: SHP_gb.idxmax(axis=1)[x])

#Simplified Code
# Passengers with missing HomePlanet and in a family with known HomePlanet
SHP_index = data[data['HomePlanet'].isna() & data['Surname'].isin(SHP_gb.index)].index
#Fill corresponding missing values
data.loc[SHP_index, 'HomePlanet'] = data.loc[SHP_index, 'Surname'].replace(SHP_gb.idxmax(axis=1))

# Print number of missing values left
print('#HomePlanet missing values before:',HP_bef)
print('#HomePlanet missing values after:',data['HomePlanet'].isna().sum())

In [ ]:
# Only 10 HomePlanet missing values left - let's look at them
data[data['HomePlanet'].isna()][['PassengerId','HomePlanet','Destination']]

#Everyone left is heading towards TRAPPIST-1e. So let's look at the joint distribution of HomePlanet and Destination.

In [ ]:
#Using Destination

#HomePlanet and Destination
# Joint distribution of HomePlanet and Destination
HPD_gb=data.groupby(['HomePlanet','Destination'])['Destination'].count().unstack().fillna(0)

# Heatmap of missing values
plt.figure(figsize=(10,4))
sns.heatmap(HPD_gb.T, annot=True, fmt='g', cmap='coolwarm')
"""
Most people heading towards TRAPPIST-1e came from Earth so it makes sense to guess they came from there. 
"""


In [ ]:
# Missing values before
HP_bef=data['HomePlanet'].isna().sum()

# Fill remaining HomePlanet missing values with Earth (if not on deck D) or Mars (if on Deck D)
data.loc[(data['HomePlanet'].isna()) & ~(data['Cabin_deck']=='D'), 'HomePlanet']='Earth'
data.loc[(data['HomePlanet'].isna()) & (data['Cabin_deck']=='D'), 'HomePlanet']='Mars'

# Print number of missing values left
print('#HomePlanet missing values before:',HP_bef)
print('#HomePlanet missing values after:',data['HomePlanet'].isna().sum())

In [ ]:
data['HomePlanet'].isna().sum()

In [ ]:
train.head()

In [ ]:
data.isna().sum()

#Long way to go to handle missing values

In [ ]:
# Create a new column 'Missing_Destination' to indicate missing values
train['Missing_Destination'] = train['Destination'].isna()

# Plot countplot with hue='Missing_Destination'
sns.countplot(data=train, x='Destination', hue='Missing_Destination')

# Display the percentage on top of each bar
total = len(train['Destination'].dropna())  # Total non-missing values
for p in plt.gca().patches:
    height = p.get_height() if not pd.isna(p.get_height()) else 0
    plt.text(p.get_x() + p.get_width() / 2., height + 0.05,
             f'{height/total:.1%}', ha='center', va='bottom')

# Show the plot
plt.show()
train.drop(['Missing_Destination'],axis=1,inplace=True)


In [ ]:
#majority of values belong to trappist-1e

#Using Destination

#HomePlanet and Destination
# Joint distribution of HomePlanet and Destination
HPD_gb=data.groupby(['HomePlanet','Destination'])['Destination'].count().unstack().fillna(0)

# Heatmap of missing values
plt.figure(figsize=(10,4))
sns.heatmap(HPD_gb.T, annot=True, fmt='g', cmap='coolwarm')
"""
Most people heading towards TRAPPIST-1e came from Earth so it makes sense to guess they came from there. 
"""

In [ ]:
des_na = train[train['Destination'].isna()]
sns.countplot(data = des_na, x='HomePlanet')

In [ ]:
Des_na_sum=data['Destination'].isna().sum()
Des_na_sum

In [ ]:
# Joint distribution of Surname and HomePlanet - Using Surname
DES_gb=data.groupby(['HomePlanet','Destination'])['Destination'].count().unstack().fillna(0)
DES_gb

In [ ]:
# Missing values before
Des_na_sum=data['Destination'].isna().sum()
DES_gb=data.groupby(['HomePlanet','Destination'])['Destination'].count().unstack().fillna(0)

# Passengers with missing Destination and in a Homeplanet
DHP_index = data[data['Destination'].isna() & data['HomePlanet'].isin(DES_gb.index)].index

# Fill corresponding missing values
data.loc[DHP_index, 'Destination'] = data.loc[DHP_index, 'HomePlanet'].replace(DES_gb.idxmax(axis=1))

# Print number of missing values left
print('#Destination missing values before:',Des_na_sum)
print('#Destination missing values after:',data['Destination'].isna().sum())

In [ ]:
data['Destination'].isna().sum()

In [ ]:
# #Destination
# # Missing values before
# D_bef=data['Destination'].isna().sum()

# # Fill missing Destination values with mode
# data.loc[(data['Destination'].isna()), 'Destination']='TRAPPIST-1e'

# # Print number of missing values left
# print('#Destination missing values before:',D_bef)
# print('#Destination missing values after:',data['Destination'].isna().sum())

In [ ]:
# Joint distribution of Group and Surname
GSN_gb=data[data['Group_size']>1].groupby(['Group','Surname'])['Surname'].size().unstack().fillna(0)
"""The majority (83%) of groups contain only 1 family. 
So let's fill missing surnames according to the majority surname in that group"""

In [ ]:
GSN_gb

In [ ]:
# Missing values before
SN_bef=data['Surname'].isna().sum()

# Passengers with missing Surname and in a group with known majority Surname
GSN_index=data[data['Surname'].isna()][(data[data['Surname'].isna()]['Group']).isin(GSN_gb.index)].index

# Fill corresponding missing values
data.loc[GSN_index,'Surname']=data.iloc[GSN_index,:]['Group'].map(lambda x: GSN_gb.idxmax(axis=1)[x])

# Print number of missing values left
print('#Surname missing values before:',SN_bef)
print('#Surname missing values after:',data['Surname'].isna().sum())

In [ ]:
# Replace NaN's with outliers (so we can use map)
data['Surname'].fillna('Unknown', inplace=True)

# Update family size feature
data['Family_size']=data['Surname'].map(lambda x: data['Surname'].value_counts()[x])

# Put NaN's back in place of outliers
data.loc[data['Surname']=='Unknown','Surname']=np.nan

# Say unknown surname means no family
data.loc[data['Family_size']>100,'Family_size']=0

In [ ]:
data['Surname'].isna().sum()

In [ ]:
data.groupby(['HomePlanet','Surname'])['Surname'].count().unstack().fillna(0)

In [ ]:
# Using HomePlanet to fill Surname
Surname_na_sum=data['Surname'].isna().sum()

SUR_gb=data.groupby(['HomePlanet','Surname'])['Surname'].count().unstack().fillna(0)

# Passengers with missing Destination and in a Homeplanet
SURHP_index = data[data['Surname'].isna() & data['HomePlanet'].isin(SUR_gb.index)].index

# Fill corresponding missing values
data.loc[SURHP_index, 'Surname'] = data.loc[SURHP_index, 'HomePlanet'].replace(SUR_gb.idxmax(axis=1))

# Print number of missing values left
print('#Destination missing values before:',Surname_na_sum)
print('#Destination missing values after:',data['Surname'].isna().sum())

In [ ]:
data.isna().sum()

In [ ]:
# Joint distribution of Group and Cabin features - all have 299 values missing
#We just want group_size > 1 ie 2-8 to handle in between missing values
GCD_gb=data[data['Group_size']>1].groupby(['Group','Cabin_deck'])['Cabin_deck'].count().unstack().fillna(0)
GCN_gb=data[data['Group_size']>1].groupby(['Group','Cabin_number'])['Cabin_number'].count().unstack().fillna(0)
GCS_gb=data[data['Group_size']>1].groupby(['Group','Cabin_side'])['Cabin_side'].count().unstack().fillna(0)

In [ ]:
data[data['Group_size']>1].groupby(['Group','Cabin_side'])['Cabin_side'].count().unstack().fillna(0)

In [ ]:
# Missing values before
CS_bef=data['Cabin_side'].isna().sum()

# Passengers with missing Cabin side and in a group with known Cabin side
GCS_index=  data.loc[data['Cabin_side'].isna() & data['Group'].isin(GCS_gb.index),:].index
# Fill corresponding missing values
data.loc[GCS_index,'Cabin_side']= data.iloc[GCS_index,:]['Group'].map(lambda x: GCS_gb.idxmax(axis=1)[x])

# Print number of missing values left
print('#Cabin_side missing values before:',CS_bef)
print('#Cabin_side missing values after:',data['Cabin_side'].isna().sum())

In [ ]:
SCS_gb = data[data['Group_size']>1].groupby(['Surname','Cabin_side'])['Cabin_side'].size().unstack().fillna(0)
SCS_gb
#Yorkland has 2.0 and 5.0 in P and S respectively meaning same surname are in 2 different cabin_side

In [ ]:
SCS_gb['P']/(SCS_gb['P']+SCS_gb['S'])

In [ ]:
# Joint distribution of Surname and Cabin side
SCS_gb=data[data['Group_size']>1].groupby(['Surname','Cabin_side'])['Cabin_side'].size().unstack().fillna(0)

# Ratio of sides - It states that how much % of family are in same cabin side. 
# Few points in between 0-1 suggest family having same name are in different Cabin_side
    #1 plausible explaination is that though Surname is same, they are not family members ie same surname but no blood relation
SCS_gb['Ratio']= SCS_gb['P']/(SCS_gb['P']+SCS_gb['S'])

# Histogram of ratio
plt.figure(figsize=(10,4))
sns.histplot(SCS_gb['Ratio'], kde=True, binwidth=0.05)
plt.title('Ratio of cabin side by Surname')

In [ ]:
# Print proportion
print('Percentage of families all on the same cabin side:', 100*np.round((SCS_gb['Ratio'].isin([0,1])).sum()/len(SCS_gb),3),'%')

# Another view of the same information
SCS_gb.head()

In [ ]:
#This shows that families tend to be on the same cabin side (and 77% of families are entirely on the same side).
# Missing values before
CS_bef=data['Cabin_side'].isna().sum()

# Drop ratio column
SCS_gb.drop('Ratio', axis=1, inplace=True)

# Passengers with missing Cabin side and in a family with known Cabin side
# SCS_index=data[data['Cabin_side'].isna()][(data[data['Cabin_side'].isna()]['Surname']).isin(SCS_gb.index)].index

# # Fill corresponding missing values
# data.loc[SCS_index,'Cabin_side']=data.iloc[SCS_index,:]['Surname'].map(lambda x: SCS_gb.idxmax(axis=1)[x])

SCS_index = data[data['Cabin_side'].isna() & data['Surname'].isin(SCS_gb.index)].index

# Fill corresponding missing values
data.loc[SCS_index, 'Cabin_side'] = data.loc[SCS_index, 'Surname'].map(SCS_gb.idxmax(axis=1))


# Drop surname (we don't need it anymore)
data.drop('Surname', axis=1, inplace=True)

# Print number of missing values left
print('#Cabin_side missing values before:',CS_bef)
print('#Cabin_side missing values after:',data['Cabin_side'].isna().sum())

In [ ]:
data.groupby(['HomePlanet','Cabin_side'])['Cabin_side'].count().unstack().fillna(0)

In [ ]:
HPCS_gb = data.groupby(['HomePlanet','Cabin_side'])['Cabin_side'].count().unstack().fillna(0)
CS_bef=data['Cabin_side'].isna().sum() #65

HPCS_index = data[data['Cabin_side'].isna() & data['HomePlanet'].isin(HPCS_gb.index)].index

# Fill corresponding missing values
data.loc[HPCS_index, 'Cabin_side'] = data.loc[HPCS_index, 'HomePlanet'].map(HPCS_gb.idxmax(axis=1))

# Print number of missing values left
print('#Cabin_side missing values before:',CS_bef)
print('#Cabin_side missing values after:',data['Cabin_side'].isna().sum())

In [ ]:
# # Missing values before
# CS_bef=data['Cabin_side'].isna().sum()

# # Fill remaining missing values with outlier
# data.loc[data['Cabin_side'].isna(),'Cabin_side']='Z'

# # Print number of missing values left
# print('#Cabin_side missing values before:',CS_bef)
# print('#Cabin_side missing values after:',data['Cabin_side'].isna().sum())

In [ ]:
data['Cabin_deck'].isna().sum()

In [ ]:
# Missing values before
CD_bef=data['Cabin_deck'].isna().sum()

# Passengers with missing Cabin deck and in a group with known majority Cabin deck
GCD_index=data[data['Cabin_deck'].isna()][(data[data['Cabin_deck'].isna()]['Group']).isin(GCD_gb.index)].index

# Fill corresponding missing values
data.loc[GCD_index,'Cabin_deck']=data.iloc[GCD_index,:]['Group'].map(lambda x: GCD_gb.idxmax(axis=1)[x])

# Print number of missing values left
print('#Cabin_deck missing values before:',CD_bef)
print('#Cabin_deck missing values after:',data['Cabin_deck'].isna().sum())

In [ ]:
# Joint distribution
data.groupby(['HomePlanet','Destination','Solo','Cabin_deck'])['Cabin_deck'].size().unstack().fillna(0)

In [ ]:
# Missing values before
CD_bef=data['Cabin_deck'].isna().sum()

#Cabin_deck --> A,B,C,D,E,F,G,T
# Fill missing values using the mode
na_rows_CD=data.loc[data['Cabin_deck'].isna(),'Cabin_deck'].index
data.loc[data['Cabin_deck'].isna(),'Cabin_deck']=data.groupby(['HomePlanet','Destination','Solo'])['Cabin_deck'].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))[na_rows_CD]

# Print number of missing values left
print('#Cabin_deck missing values before:',CD_bef)
print('#Cabin_deck missing values after:',data['Cabin_deck'].isna().sum())

In [ ]:
# Create a scatter plot
scatter_plot = sns.scatterplot(
    x=data['Cabin_number'],
    y=data['Group'],
    hue=data.loc[data['Cabin_number'].notnull(),'Cabin_deck'],
    palette='tab10'
)

# Add legend
scatter_plot.legend(title='Cabin_deck')

# Show the plot
plt.show()

In [ ]:
data.loc[(data['Cabin_number'].notnull()) & (data['Cabin_deck']=='A'),'Group']

In [ ]:
 data.loc[(data['Cabin_number'].isna()) & (data['Cabin_deck']=="A"),'Group']

In [ ]:
# Missing values before
CN_bef=data['Cabin_number'].isna().sum()

# Extrapolate linear relationship on a deck by deck basis
for deck in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
    # Features and labels
    X_CN= data.loc[(data['Cabin_number'].notnull()) & (data['Cabin_deck']==deck),'Group']
    y_CN= data.loc[(data['Cabin_number'].notnull()) & (data['Cabin_deck']==deck),'Cabin_number']
    
    X_test_CN= data.loc[(data['Cabin_number'].isna()) & (data['Cabin_deck']==deck),'Group']

    # Linear regression
    lr= LinearRegression()
    lr.fit(X_CN.values.reshape(-1, 1), y_CN)
    preds_CN=lr.predict(X_test_CN.values.reshape(-1, 1))
    
    # Fill missing values with predictions
    data.loc[(data['Cabin_number'].isna()) & (data['Cabin_deck']==deck),'Cabin_number']=preds_CN.astype(int)

# Print number of missing values left
print('#Cabin_number missing values before:',CN_bef)
print('#Cabin_number missing values after:',data['Cabin_number'].isna().sum())

In [ ]:
data.head()

In [ ]:
# One-hot encode cabin regions
data['Cabin_region1']=(data['Cabin_number']<300).astype(int)
data['Cabin_region2']=((data['Cabin_number']>=300) & (data['Cabin_number']<600)).astype(int)
data['Cabin_region3']=((data['Cabin_number']>=600) & (data['Cabin_number']<900)).astype(int)
data['Cabin_region4']=((data['Cabin_number']>=900) & (data['Cabin_number']<1200)).astype(int)
data['Cabin_region5']=((data['Cabin_number']>=1200) & (data['Cabin_number']<1500)).astype(int)
data['Cabin_region6']=((data['Cabin_number']>=1500) & (data['Cabin_number']<1800)).astype(int)
data['Cabin_region7']=(data['Cabin_number']>=1800).astype(int)

In [ ]:
# Missing values before
V_bef=data['VIP'].isna().sum()

# Fill missing values with mode
data.loc[data['VIP'].isna(),'VIP']=False

# Print number of missing values left
print('#VIP missing values before:',V_bef)
print('#VIP missing values after:',data['VIP'].isna().sum())

In [ ]:
data.isna().sum()

In [ ]:
#Age varies across many features like HomePlanet, group size, expenditure and cabin deck, 
#so we will impute missing values according to the median of these subgroups.

# Joint distribution
data.groupby(['HomePlanet','No_spending','Solo','Cabin_deck'])['Age'].median().unstack().fillna(0)


In [ ]:
# Missing values before
A_bef=data[exp_feats].isna().sum().sum()

# Fill missing values using the median
na_rows_A=data.loc[data['Age'].isna(),'Age'].index
data.loc[data['Age'].isna(),'Age']=data.groupby(['HomePlanet','No_spending','Solo','Cabin_deck'])['Age'].transform(lambda x: x.fillna(x.median()))[na_rows_A]

# Print number of missing values left
print('#Age missing values before:',A_bef)
print('#Age missing values after:',data['Age'].isna().sum())

In [ ]:
# Update age group feature
data.loc[data['Age']<=12,'Age_group']='Age_0-12'
data.loc[(data['Age']>12) & (data['Age']<18),'Age_group']='Age_13-17'
data.loc[(data['Age']>=18) & (data['Age']<=25),'Age_group']='Age_18-25'
data.loc[(data['Age']>25) & (data['Age']<=30),'Age_group']='Age_26-30'
data.loc[(data['Age']>30) & (data['Age']<=50),'Age_group']='Age_31-50'
data.loc[data['Age']>50,'Age_group']='Age_51+'

In [ ]:
# Joint distribution
data.groupby(['No_spending','CryoSleep'])['CryoSleep'].size().unstack().fillna(0)

In [ ]:
# Missing values before
CSL_bef=data['CryoSleep'].isna().sum()

# Fill missing values using the mode
na_rows_CSL=data.loc[data['CryoSleep'].isna(),'CryoSleep'].index
data.loc[data['CryoSleep'].isna(),'CryoSleep']=data.groupby(['No_spending'])['CryoSleep'].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))[na_rows_CSL]

# Print number of missing values left
print('#CryoSleep missing values before:',CSL_bef)
print('#CryoSleep missing values after:',data['CryoSleep'].isna().sum())

In [ ]:
#This one makes a lot of sense. We don't expect people in CryoSleep to be able to spend anything.
print('Maximum expenditure of passengers in CryoSleep:',data.loc[data['CryoSleep']==True,exp_feats].sum(axis=1).max())

In [ ]:
# Missing values before
E_bef=data[exp_feats].isna().sum().sum()

# CryoSleep has no expenditure
for col in exp_feats:
    data.loc[(data[col].isna()) & (data['CryoSleep']==True), col]=0

# Print number of missing values left
print('#Expenditure missing values before:',E_bef)
print('#Expenditure missing values after:',data[exp_feats].isna().sum().sum())

In [ ]:
data['Expenditure'].mean(),  data['Expenditure'].median(),
#This states that those not in cryo sleep are spending way too much causing outliers

In [ ]:
sns.boxplot(data = data, x= 'Expenditure',hue='CryoSleep')
plt.title('Expenditure BoxPlot')

In [ ]:
"""
Expenditure varies across many features but we will only impute missing values using HomePlanet, 
Solo and Age group to prevent overfitting. We will also use the mean instead of the median because a 
large proportion of passengers don't spend anything and median usually comes out as 0. 
Note how under 12 age don't spend anything suggesting they have no spending power which is obvious.
"""

# Joint distribution
data.groupby(['HomePlanet','Solo','Age_group'])['Expenditure'].mean().unstack().fillna(0)

In [ ]:
# Missing values before
E_bef=data[exp_feats].isna().sum().sum()

# Fill remaining missing values using the median
for col in exp_feats:
    na_rows=data.loc[data[col].isna(),col].index
    data.loc[data[col].isna(),col]=data.groupby(['HomePlanet','Solo','Age_group'])[col].transform(lambda x: x.fillna(x.mean()))[na_rows]
    
# Print number of missing values left
print('#Expenditure missing values before:',E_bef)
print('#Expenditure missing values after:',data[exp_feats].isna().sum().sum())

In [ ]:
# Update expenditure and no_spending
data['Expenditure']=data[exp_feats].sum(axis=1)
data['No_spending']=(data['Expenditure']==0).astype(int)

In [ ]:
data.isna().sum()

In [ ]:
label = LabelEncoder()
data['HomePlanet'] = label.fit_transform(data['HomePlanet'])
data['CryoSleep'] = label.fit_transform(data['CryoSleep'])
data['Destination'] = label.fit_transform(data['Destination'])
data['VIP'] = label.fit_transform(data['VIP'])
data['Age_group'] = label.fit_transform(data['Age_group'])
data['Cabin_deck'] = label.fit_transform(data['Cabin_deck'])
data['Cabin_side'] = label.fit_transform(data['Cabin_side'])

In [ ]:
data

In [ ]:
#Preprocessing
# Train and test
X=data[data['PassengerId'].isin(train['PassengerId'].values)].copy()
X_test=data[data['PassengerId'].isin(test['PassengerId'].values)].copy()

# Drop qualitative/redundant/collinear/high cardinality features
X.drop(['PassengerId', 'Group', 'Age_group', 'Cabin_number'], axis=1, inplace=True)
X_test.drop(['PassengerId', 'Group', 'Age_group', 'Cabin_number'], axis=1, inplace=True)

In [ ]:
len(X), len(y)

In [ ]:
# Train-validation split
X_train, X_valid, y_train, y_valid = train_test_split(X,y,stratify=y,train_size=0.8,test_size=0.2,random_state=42)

In [ ]:
#Modelling

# Put models in a dictionary
models = {"KNN": KNeighborsClassifier(),
          "Logistic Regression": LogisticRegression(), 
          "Random Forest": RandomForestClassifier()}

# Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    # Random seed for reproducible results
    np.random.seed(42)
    # Make a list to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

In [ ]:
model_scores = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_valid,
                             y_train=y_train,
                             y_test=y_valid)
model_scores

In [ ]:
#Model Comparison
model_compare = pd.DataFrame(model_scores, index=['accuracy'])
model_compare.T.plot.bar();

In [ ]:
"""
Hyperparameter tuning - Each model you use has a series of dials you can turn to dictate how they perform. Changing these values may increase or decrease model performance.
Feature importance - If there are a large amount of features we're using to make predictions, do some have more importance than others? For example, for predicting heart disease, which is more important, sex or age?
Confusion matrix - Compares the predicted values with the true values in a tabular way, if 100% correct, all values in the matrix will be top left to bottom right (diagnol line).
Cross-validation - Splits your dataset into multiple parts and train and tests your model on each part and evaluates performance as an average.
Precision - Proportion of true positives over total number of samples. Higher precision leads to less false positives.
Recall - Proportion of true positives over total number of true positives and false negatives. Higher recall leads to less false negatives.
F1 score - Combines precision and recall into one metric. 1 is best, 0 is worst.
Classification report - Sklearn has a built-in function called classification_report() which returns some of the main classification metrics such as precision, recall and f1-score.
ROC Curve - Receiver Operating Characterisitc is a plot of true positive rate versus false positive rate.
Area Under Curve (AUC) - The area underneath the ROC curve. A perfect model achieves a score of 1.0.
"""

In [ ]:
#Using RandomizedSearchCV on Logistic Regression and Random Forest

In [ ]:
# Setup random seed

# Different LogisticRegression hyperparameters
log_reg_grid = {"C": np.logspace(-4, 4, 20),
                "solver": ["liblinear"], 
                'penalty': ['l1','l2'],
                'max_iter': [50, 100, 150]
               }
np.random.seed(42)

# Setup random hyperparameter search for LogisticRegression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model
rs_log_reg.fit(X_train, y_train)
lr_bp = rs_log_reg.best_params_
print(lr_bp) #{'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 100, 'C': 0.08858667904100823}
rs_log_reg.score(X_valid, y_valid) #'Logistic Regression': 0.7843588269120184 earlier

In [ ]:
# Different RandomForestClassifier hyperparameters
rf_grid = {"n_estimators": [50, 100, 150, 200, 250, 300],
           "max_depth": [4, 6, 8, 10, 12],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2)}


# Setup random seed
np.random.seed(42)

# Setup random hyperparameter search for RandomForestClassifier
rs_rf = RandomizedSearchCV(RandomForestClassifier(),
                           param_distributions=rf_grid,
                           cv=5,
                           n_iter=20,
                           verbose=True)

# Fit random hyperparameter search model
rs_rf.fit(X_train, y_train) #{'n_estimators': 300, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_depth': 10}
rf_bp = rs_rf.best_params_
print(rf_bp)
rs_rf.score(X_valid, y_valid) #'Random Forest': 0.8067855089131685

In [ ]:
# Check best hyperparameters
rs_rf.best_params_

In [ ]:
#GridSearchCV will take a long time 
#Evaluating a classification model, beyond accuracy
#Using default RandomForest with oob_score

model1 = RandomForestClassifier(oob_score=True)
model1.fit(X_train,y_train)
model1.score(X_valid, y_valid)

In [ ]:
# Make preidctions on test data
y_preds = model1.predict(X_valid)

#Using ROC Curve and AUC Scores
from sklearn.metrics import RocCurveDisplay 

RocCurveDisplay.from_estimator(estimator=model1, X=X_valid, y=y_valid)

In [ ]:
#plotting Confusion matrix

sns.set(font_scale=1.5) # Increase font size

def plot_conf_mat(y_valid, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_valid, y_preds),
                     annot=True, # Annotate the boxes
                      fmt='d',
                     cbar=False)
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    
plot_conf_mat(y_valid, y_preds)

In [ ]:
model1.oob_score_  #79% of time it classifiers the data properly on unseen data

In [ ]:
#Classification report
# Show classification report
"""
Precision - Indicates the proportion of positive identifications (model predicted class 1) which were actually correct. A model which produces no false positives has a precision of 1.0.
Recall - Indicates the proportion of actual positives which were correctly classified. A model which produces no false negatives has a recall of 1.0.
F1 score - A combination of precision and recall. A perfect model achieves an F1 score of 1.0.
Support - The number of samples each metric was calculated on.
Accuracy - The accuracy of the model in decimal form. Perfect accuracy is equal to 1.0.
Macro avg - Short for macro average, the average precision, recall and F1 score between classes. Macro avg doesn’t class imbalance into effort, so if you do have class imbalances, pay attention to this metric.
Weighted avg - Short for weighted average, the weighted average precision, recall and F1 score between classes. Weighted means each metric is calculated with respect to how many samples there are in each class. This metric will favour the majority class (e.g. will give a high value when one class out performs another due to having more samples).

"""
print(classification_report(y_valid, y_preds))

In [ ]:
model1.get_params() #

In [ ]:
# Import cross_val_score is an alternate to train_test_split
from sklearn.model_selection import cross_val_score

# Instantiate best model with best hyperparameters (found with GridSearchCV)
clf = RandomForestClassifier()

# Cross-validated accuracy score
cv_acc = cross_val_score(clf,X,y,
                         cv=5, # 5-fold cross-validation
                         scoring="f1") # accuracy, precision, recall,f1
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
#We have Random Forest Classifer Model that we can use, let's try some other models

#Modelling

# Put models in a dictionary
models = {"SVC" : SVC(random_state=0, probability=True),
          "NaiveBayes": GaussianNB(), 
          "XGBoost" : XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss')}

# Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    # Random seed for reproducible results
    np.random.seed(42)
    # Make a list to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

model_scores2 = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_valid,
                             y_train=y_train,
                             y_test=y_valid)
model_scores2

In [ ]:
#Model Comparison
model_compare = pd.DataFrame(model_scores2, index=['f1'])
model_compare.T.plot.bar();

In [ ]:
# We choose SVC and XGBoost for hyperparameter tuning

SVC_hyp = {'C': [0.25, 0.5, 0.75, 1, 1.25, 1.5],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto']}

# Setup random seed
np.random.seed(42)

# Setup random hyperparameter search for SVC
rs_svg = RandomizedSearchCV(SVC(random_state=0, probability=True),
                                param_distributions=SVC_hyp,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model
rs_svg.fit(X_train, y_train)
svc_best_params = rs_svg.best_params_
print(svc_best_params)
rs_svg.score(X_valid, y_valid)

In [ ]:

xgboost_hyp = {'n_estimators': [50, 100, 150, 200],
        'max_depth': [4, 8, 12],
        'learning_rate': [0.05, 0.1, 0.15]}

# Setup random seed
np.random.seed(42)

# Setup random hyperparameter search for LogisticRegression
rs_xgb = RandomizedSearchCV(XGBClassifier(random_state=0, use_label_encoder=False, eval_metric='logloss'),
                                param_distributions=xgboost_hyp,
                                cv=5,
                                n_iter=20,
                                verbose=True)

# Fit random hyperparameter search model
rs_xgb.fit(X_train, y_train)
xgb_best_params = rs_xgb.best_params_
print(xgb_best_params)
rs_xgb.score(X_valid, y_valid)

In [ ]:
#Evaluating

# Check best hyperparameters
model2 = XGBClassifier(random_state=42, 
                       use_label_encoder=False, 
                       eval_metric='logloss',
                       n_estimators=100,
                       max_depth=4,
                       learning_rate=0.1)
model2.fit(X_train,y_train)
model2.score(X_valid, y_valid)

In [ ]:
# Make preidctions on test data
y_preds2 = model2.predict(X_valid)

#Using ROC Curve and AUC Scores
from sklearn.metrics import RocCurveDisplay 

RocCurveDisplay.from_estimator(estimator=model2, X=X_valid, y=y_valid)

In [ ]:
#plotting Confusion matrix

sns.set(font_scale=1.5) # Increase font size

def plot_conf_mat(y_valid, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_valid, y_preds),
                     annot=True, # Annotate the boxes
                      fmt='d',
                     cbar=False)
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    
plot_conf_mat(y_valid, y_preds2)

In [ ]:
print(classification_report(y_valid, y_preds2))

In [ ]:
model2.get_params() #

In [ ]:
# Import cross_val_score is an alternate to train_test_split
from sklearn.model_selection import cross_val_score

# Instantiate best model with best hyperparameters
clf2 = XGBClassifier(random_state=42, 
                       use_label_encoder=False, 
                       eval_metric='logloss',
                       n_estimators=100,
                       max_depth=4,
                       learning_rate=0.1)

# Cross-validated accuracy score
cv_acc2 = cross_val_score(clf2,X,y,
                         cv=5, # 5-fold cross-validation
                         scoring="f1") # accuracy, precision, recall,f1
cv_acc2

In [ ]:
np.mean(cv_acc2) #XGBoost is a best option then RandomForestClassification

In [ ]:
#Modelling
# Put models in a dictionary
models = {"LGBM" : LGBMClassifier(random_state=0),
            "CatBoost" : CatBoostClassifier(random_state=0, verbose=False),
            "GradientBoost": GradientBoostingClassifier(random_state=0)}

# Create function to fit and score models
def fit_and_score(models, X_train, X_test, y_train, y_test):
    # Random seed for reproducible results
    np.random.seed(42)
    # Make a list to keep model scores
    model_scores = {}
    # Loop through models
    for name, model in models.items():
        # Fit the model to the data
        model.fit(X_train, y_train)
        # Evaluate the model and append its score to model_scores
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

model_scores3 = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_valid,
                             y_train=y_train,
                             y_test=y_valid)
model_scores3

In [ ]:
#Model Comparison
model_compare = pd.DataFrame(model_scores3, index=['f1'])
model_compare.T.plot.bar();

In [ ]:
# We choose LGMB and CatBoost for hyperparameter tuning

# Define the hyperparameter grid
boosted_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [4, 8, 12],
    'learning_rate': [0.05, 0.1, 0.15],
    'num_leaves': [10, 20, 30, 40], 
}


# Setup random hyperparameter search for LightGBM
rs_lgbm = RandomizedSearchCV(
    LGBMClassifier(random_state=0, verbose=0),
    param_distributions=boosted_grid,
    cv=5,
    n_iter=20,
    verbose=0,
    scoring='accuracy',  # Use an appropriate scoring metric
    random_state=42,
    n_jobs=-1  # Use multiple cores for parallel processing
)

# Fit random hyperparameter search model
rs_lgbm.fit(X_train, y_train)
lgbm_best_params = rs_lgbm.best_params_

# Evaluate the model on the validation set
accuracy = rs_lgbm.score(X_valid, y_valid)
print(f"Best Hyperparameters: {lgbm_best_params}")
print(f"Validation Accuracy: {accuracy}")

# Now, you can use the best_params_ to create a final LightGBM model for further evaluation
final_lgbm_model = LGBMClassifier(**lgbm_best_params, random_state=0, verbose=0)
final_lgbm_model.fit(X_train, y_train)
final_accuracy = final_lgbm_model.score(X_valid, y_valid)
print(f"Final Model Validation Accuracy: {final_accuracy}")


# Best Hyperparameters: {'num_leaves': 20, 'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.1}
# Validation Accuracy: 0.8159861989649224
# Final Model Validation Accuracy: 0.8159861989649224

In [ ]:
# Make preidctions on test data
y_preds3 = final_lgbm_model.predict(X_valid)

#Using ROC Curve and AUC Scores
from sklearn.metrics import RocCurveDisplay 

RocCurveDisplay.from_estimator(estimator=final_lgbm_model, X=X_valid, y=y_valid)

In [ ]:
#plotting Confusion matrix

sns.set(font_scale=1.5) # Increase font size

def plot_conf_mat(y_valid, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_valid, y_preds),
                     annot=True, # Annotate the boxes
                      fmt='d',
                     cbar=False)
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    
plot_conf_mat(y_valid, y_preds3)

In [ ]:
print(classification_report(y_valid, y_preds3))

In [ ]:
final_lgbm_model.get_params() #

In [ ]:
# Import cross_val_score is an alternate to train_test_split
from sklearn.model_selection import cross_val_score

# Instantiate best model with best hyperparameters
#clf3 = LGBMClassifier(*lgbm_best_params, random_state=0, verbose=0)

# Cross-validated accuracy score
cv_acc3 = cross_val_score(final_lgbm_model,X,y,
                         cv=5, # 5-fold cross-validation
                         scoring="f1") # accuracy, precision, recall,f1
cv_acc3

In [ ]:
np.mean(cv_acc3)

In [ ]:
# We choose LGMB and CatBoost for hyperparameter tuning

# Define the hyperparameter grid
boosted_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [4, 8, 12],
    'learning_rate': [0.05, 0.1, 0.15] 
}


# Setup random hyperparameter search for LightGBM
rs_cat = RandomizedSearchCV(
    CatBoostClassifier(random_state=0, verbose=0),
    param_distributions=boosted_grid,
    cv=5,
    n_iter=20,
    verbose=0,
    scoring='accuracy',  # Use an appropriate scoring metric
    random_state=42,
    n_jobs=-1  # Use multiple cores for parallel processing
)

# Fit random hyperparameter search model
rs_cat.fit(X_train, y_train)
cat_best_params = rs_cat.best_params_
# Evaluate the model on the validation set
accuracy = rs_cat.score(X_valid, y_valid)
print(f"Best Hyperparameters: {cat_best_params}")
print(f"Validation Accuracy: {accuracy}")

# Now, you can use the best_params_ to create a final LightGBM model for further evaluation
final_catboost_model = CatBoostClassifier(**cat_best_params, random_state=0, verbose=0)
final_catboost_model.fit(X_train, y_train)
final_accuracy_cat = final_catboost_model.score(X_valid, y_valid)
print(f"Final Model Validation Accuracy: {final_accuracy_cat}")

In [ ]:
# Make preidctions on test data
y_preds4 = final_catboost_model.predict(X_valid)

#Using ROC Curve and AUC Scores
from sklearn.metrics import RocCurveDisplay 

RocCurveDisplay.from_estimator(estimator=final_catboost_model, X=X_valid, y=y_valid)

In [ ]:
#plotting Confusion matrix

sns.set(font_scale=1.5) # Increase font size

def plot_conf_mat(y_valid, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_valid, y_preds),
                     annot=True, # Annotate the boxes
                      fmt='d',
                     cbar=False)
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    
plot_conf_mat(y_valid, y_preds4)

In [ ]:
print(classification_report(y_valid, y_preds4))

In [ ]:
final_catboost_model.get_params() #

In [ ]:
# Import cross_val_score is an alternate to train_test_split
from sklearn.model_selection import cross_val_score

# Instantiate best model with best hyperparameters
clf4 = CatBoostClassifier(**cat_best_params, random_state=0, verbose=0)

# Cross-validated accuracy score
cv_acc4 = cross_val_score(clf4,X,y,
                         cv=5, # 5-fold cross-validation
                         scoring="f1") # accuracy, precision, recall,f1
cv_acc4

In [ ]:
np.mean(cv_acc4)

In [ ]:
#We go for Light GBM as our main model, we will add and remove features to increase the accuracy

In [ ]:
# Define the hyperparameter grid and improving our model further
boosted_grid = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    'reg_lambda': [0, 0.1, 0.5, 1.0],
    'min_child_samples': [5, 10, 20],
    'num_leaves': [20, 31, 40, 50]
}


# Setup random hyperparameter search for LightGBM
rs_lgbm = RandomizedSearchCV(
    LGBMClassifier(random_state=0, verbose=0),
    param_distributions=boosted_grid,
    cv=5,
    n_iter=20,
    verbose=0,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1  
)

# Fit random hyperparameter search model
rs_lgbm.fit(X_train, y_train)
lgbm_best_params = rs_lgbm.best_params_

# Evaluate the model on the validation set
accuracy = rs_lgbm.score(X_valid, y_valid)
print(f"Best Hyperparameters: {lgbm_best_params}")
print(f"Validation Accuracy: {accuracy}")

# Now, you can use the best_params_ to create a final LightGBM model for further evaluation
final_lgbm_model = LGBMClassifier(**lgbm_best_params, random_state=0, verbose=0)
final_lgbm_model.fit(X_train, y_train)
final_accuracy = final_lgbm_model.score(X_valid, y_valid)
print(f"Final Model Validation Accuracy: {final_accuracy}")


# Best Hyperparameters: {'num_leaves': 20, 'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.1}
# Validation Accuracy: 0.8159861989649224
# Final Model Validation Accuracy: 0.8159861989649224

In [ ]:
import time
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Number of folds in cross-validation
FOLDS = 10

preds = np.zeros(len(X_test))
start = time.time()

# 10-fold cross-validation
cv = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

score = 0

for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    # Get training and validation sets
    X_train, X_valid = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[val_idx]

    # Train model
    clf = final_lgbm_model
    clf.fit(X_train, y_train)

    # Make predictions and measure accuracy
    preds += clf.predict_proba(X_test)[:, 1]
    score += clf.score(X_valid, y_valid)

# Average accuracy
score = score / FOLDS

# Stop timer
stop = time.time()

# Print accuracy and time
print('Model:', 'LightGBM')
print('Average validation accuracy:', np.round(100 * score, 2))
print('Training time (mins):', np.round((stop - start) / 60, 2))
print('')

# Ensemble predictions
preds = preds / FOLDS

In [ ]:
# Proportion (in test set) we get from rounding
print(np.round(100*np.round(preds).sum()/len(preds),2))

In [ ]:
# Proportion of predicted positive (transported) classes
def preds_prop(preds_arr, thresh):
    pred_classes=(preds_arr>=thresh).astype(int)
    return pred_classes.sum()/len(pred_classes)

# Plot proportions across a range of thresholds
def plot_preds_prop(preds_arr):
    # Array of thresholds
    T_array=np.arange(0,1,0.001)
    
    # Calculate proportions
    prop=np.zeros(len(T_array))
    for i, T in enumerate(T_array):
        prop[i]=preds_prop(preds_arr, T)
        
    # Plot proportions
    plt.figure(figsize=(10,4))
    plt.plot(T_array, prop)
    target_prop=0.519         # Experiment with this value
    plt.axhline(y=target_prop, color='r', linestyle='--')
    plt.text(-0.02,0.45,f'Target proportion: {target_prop}', fontsize=14)
    plt.title('Predicted target distribution vs threshold')
    plt.xlabel('Threshold')
    plt.ylabel('Proportion')
    
    # Find optimal threshold (the one that leads to the proportion being closest to target_prop)
    T_opt=T_array[np.abs(prop-target_prop).argmin()]
    print('Optimal threshold:', T_opt)
    return T_opt
    
T_opt=plot_preds_prop(preds)

In [ ]:
# Classify test set using optimal threshold
preds_tuned=(preds>=T_opt).astype(int)

In [ ]:
# Sample submission (to get right format)
sub=pd.read_csv('../input/spaceship-titanic/sample_submission.csv')

# Add predictions
sub['Transported']=preds_tuned

# Replace 0 to False and 1 to True
sub=sub.replace({0:False, 1:True})

# Prediction distribution
plt.figure(figsize=(6,6))
sub['Transported'].value_counts().plot.pie(explode=[0.1,0.1], autopct='%1.1f%%', shadow=True, textprops={'fontsize':16}).set_title("Prediction distribution")

In [ ]:
# Output to csv
sub.to_csv('submission.csv', index=False)

In [ ]:
final = pd.read_csv('/kaggle/working/submission.csv')
final